In [1]:
#!pip install requests beautifulsoup4

In [2]:
import sys
import bs4
print("Versión de Beautiful Soup:", bs4.__version__)
print("Versión de Python:", sys.version)

Versión de Beautiful Soup: 4.8.2
Versión de Python: 3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]


# Primer scrapeo

El proceso de scrapear una página web es parecido a lo que hace un humano cuando quiere buscar algo en Internet, la diferencia es que en lugar de ver el contenido presentado por el navegador, el programa analiza y selecciona el código fuente generalmente programado en  HTML y JavaScript.

El primer paso es por tanto seleccionar una página web para hacer el scraping y descargarla. Para ello, haremos peticiones de HTTP mediante `requests`:

In [31]:
#Escrapear es como buscar info en la web usualmente programado en formatos HTML y JavaScript
import requests

#El primer paso es por tanto seleccionar una página web para hacer el scraping y descargarla. Ya sabemos cómo hacer peticiones HTTP mediante `requests` así que vamos a a hacer una petición de la web de ejemplo por excelencia:
req = requests.get("https://example.com")

El caso es que como respuesta a la petición se nos ha develto la página y podemos ver su código fuente en crudo:

In [32]:
print(req.text)

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

Aquí tenemos un documento HTML bien estructurado con sus etiquetas.

Cuando el navegador interpreta estas etiquetas que se abren y se cierran, con sus atributos y contenidos genera lo que se conoce como **DOM** (*Document Object Model*), una interfaz de programación para documentos HTML y XML que en esencia es como un árbol ramificado de  componentes padres e hijos. El padre de todo es `html`, que tiene dos hijos `head` y `body`, el primero contiene el `title` y los metadatos, el otro el contenido de la página, una capa `div` que a su vez tiene una cabecera `h1` y unos parágrados `p`.

Pues bien, la biblioteca `BeautifulSoup` lo que hace es generar su propia estructura parecida a la interfaz **DOM** pero en Python, creando un árbol con los elementos del documento. Básicamente le pasamos un documento HTML en crudo y ella lo transforma en un objeto dinámico con el que podemos interactuar:

In [33]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(req.text)

print(soup)

<!DOCTYPE html>
<html>
<head>
<title>Example Domain</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>
</head>
<body>
<div>
<h1>Example Domain</h1>
<p>This domain is for use in illustrative examples

A simple vista parece lo mismo, pero ahora podemos hacer algo como esto para consultar el título de la página:

In [34]:
soup.select("title")

[<title>Example Domain</title>]

Esto que nos devuelve es un objeto, veamos tu tipo:

In [35]:
type(soup.select("title"))

bs4.element.ResultSet

Es un conjunto de resultados que contiene los tags que concuerdan con el nombre `title`, por tanto es una lista.

Veamos qué tipo tiene ese primer valor del conjunto:

In [36]:
type(soup.select("title")[0])

bs4.element.Tag

Como ven es un `Tag` y éste contiene diferentes métodos, como por ejemplo `getText()` para recuperar su contenido:

In [37]:
soup.select("title")[0].getText()

'Example Domain'

Podemos recuperar otros elementos esenciales como la cabecera o los párrafos:

In [38]:
soup.select("h1")

[<h1>Example Domain</h1>]

In [39]:
soup.select("p")

[<p>This domain is for use in illustrative examples in documents. You may use this
     domain in literature without prior coordination or asking for permission.</p>,
 <p><a href="https://www.iana.org/domains/example">More information...</a></p>]

El segundo párrafo contiene a su vez un enlace, podemos acceder de forma anidada:

In [40]:
# Seleccionar del segundo párrafo el primer enlace 
a = soup.select("p")[1].select("a")[0]

# Mostrar su contenido
a.getText()

'More information...'

Las etiquetas tienen valores especiales llamados atributos, como la dirección `href` de un enlace. 

Estos se almacenan como un diccionario del objeto, es muy cómodo acceder a ellos:

In [41]:
# Atributo con la dirección del enlace
a['href']

'https://www.iana.org/domains/example'

Estos valores están mapeados del diccionario `attrs`:

In [42]:
a.attrs.items()

dict_items([('href', 'https://www.iana.org/domains/example')])

Siguiendo esta lógica podemos programar un script que recupere todos los atributos de los metadatos:

In [43]:
for meta in soup.select("meta"):
    for atributo, valor in meta.attrs.items():
        print(f"{atributo}: {valor}")

charset: utf-8
http-equiv: Content-type
content: text/html; charset=utf-8
name: viewport
content: width=device-width, initial-scale=1


# Uso de selectores

Los selectores son una técnica para especificar de forma más concreta qué elementos recuperar del árbol:

| Selector                 | Descripción                      | 
|--------------------------|----------------------------------| 
| Etiquetas                | `soup.select("tag")`             | 
| Identificadores          | `soup.select("#id")`             | 
| Clases                   | `soup.select(".class")`          | 
| Atributos                | `soup.select("tag[attr)`         |  
| Etiquetas en etiquetas   | `soup.select("parent child")`    | 

Vamos a scrapear [la página de Python](https://es.wikipedia.org/wiki/Python) en Wikipedia para poner en práctica algunos de estos selectores:

In [44]:
import requests
from bs4 import BeautifulSoup

req = requests.get("https://web.archive.org/web/20220722211457/https://es.wikipedia.org/wiki/Python")
soup = BeautifulSoup(req.text)

title = soup.select("title")[0].getText()
print(title)

Python - Wikipedia, la enciclopedia libre


Haciendo **clic derecho inspeccionar en cualquier navegador** podemos ver el código fuente mientras seleccionamos los elementos de la página.

Por ejemplo, en el primer parágrafo del documento encontramos un resumen del artículo:


In [45]:
resumen = soup.select("p")[0].getText()
print(resumen)

Python es un lenguaje de alto nivel de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código, se utiliza para desarrollar aplicaciones de todo tipo, ejemplos: Instagram, Netflix, Spotify, Panda 3D, entre otros.[2]​ Se trata de un lenguaje de programación multiparadigma, ya que soporta parcialmente la orientación a objetos, programación imperativa y, en menor medida[¿cuál?], programación funcional. Es un lenguaje interpretado, dinámico y multiplataforma.



## Scrapeando el índice

La Wikipedia tiene lo que se conoce como **Tabla de contenidos**, un índice del documento. 

Según el inspector su etiqueta abre con este tag:

```html
<div id="toc" class="toc" role="navigation" aria-labelledby="mw-toc-heading">
```

Es una capa `div` pero tiene un identificador `id` así que podemos usar un selector y luego podemos extraer sus enlaces `a`:

In [46]:
toc = soup.select("#toc")[0]

for a in toc.select("a"):
    print(a.getText())

1 Historia
2 Características y paradigmas
3 Filosofía
4 Modo interactivo
5 Elementos del lenguaje y sintaxis
5.1 Comentarios
5.2 Variables
5.3 Tipos de datos
5.4 Condicionales
5.5 Bucle for
5.6 Bucle while
5.7 Listas y Tuplas
5.8 Diccionarios
5.9 Sentencia Switch Case
5.9.1 Usando if, elif, else
5.9.2 Usando diccionario
5.10 Conjuntos
5.11 Listas por comprensión
5.12 Funciones
5.13 Clases
5.14 Módulos
5.14.1 Instalación de módulos (pip)
5.14.2 Interfaz al sistema operativo
5.14.3 Comodines de archivos
5.14.4 Argumentos de línea de órdenes
5.14.5 Matemática
5.14.6 Fechas y Tiempos
5.14.7 Módulo Turtle
6 Sistema de objetos
7 Biblioteca estándar
8 Implementaciones
9 Incidencias
10 Véase también
11 Referencias
12 Bibliografía
13 Enlaces externos


Con una expresión regular podemos mostrar únicamente los enlaces de primer nivel:

In [47]:
import re

for a in toc.select("a"):
    text = a.getText()
    if re.match(r"\d+ ", text):
        print(text)

1 Historia
2 Características y paradigmas
3 Filosofía
4 Modo interactivo
5 Elementos del lenguaje y sintaxis
6 Sistema de objetos
7 Biblioteca estándar
8 Implementaciones
9 Incidencias
10 Véase también
11 Referencias
12 Bibliografía
13 Enlaces externos


In [48]:
import re

for a in toc.select("a"):
    text = a.getText()
    if re.match(r"\d+ ", text):
        print(text)
    elif re.match(r"\d+.\d+ ", text):
        print(" ", text)
    elif re.match(r"\d+.\d+.\d+ ", text):
        print("   ", text)

1 Historia
2 Características y paradigmas
3 Filosofía
4 Modo interactivo
5 Elementos del lenguaje y sintaxis
  5.1 Comentarios
  5.2 Variables
  5.3 Tipos de datos
  5.4 Condicionales
  5.5 Bucle for
  5.6 Bucle while
  5.7 Listas y Tuplas
  5.8 Diccionarios
  5.9 Sentencia Switch Case
    5.9.1 Usando if, elif, else
    5.9.2 Usando diccionario
  5.10 Conjuntos
  5.11 Listas por comprensión
  5.12 Funciones
  5.13 Clases
  5.14 Módulos
    5.14.1 Instalación de módulos (pip)
    5.14.2 Interfaz al sistema operativo
    5.14.3 Comodines de archivos
    5.14.4 Argumentos de línea de órdenes
    5.14.5 Matemática
    5.14.6 Fechas y Tiempos
    5.14.7 Módulo Turtle
6 Sistema de objetos
7 Biblioteca estándar
8 Implementaciones
9 Incidencias
10 Véase también
11 Referencias
12 Bibliografía
13 Enlaces externos


## Scrapeando la caja de información

Otro elemento interesante que encontramos en el artículo es la caja de información arriba a la derecha, si la insepeccionamos veremos que es una tabla con la clase `infobox`: 


```html
<table class="infobox" style="width:22.7em; line-height: 1.4em; text-align:left; padding:.23em;">
```

En el lenguaje HTML las tablas se forman a partir de etiquetas con filas `tr` que contienen cabeceras `th` o celdas con datos `td`.

Podemos empezar recorriendo las filas a ver qué encontramos:

In [49]:
tr_tags = soup.select(".infobox tr")

for tr_tag in tr_tags: #para cada fila en las filas extraemos el texto
    print(tr_tag.getText())

Python


Desarrollador(es)

Python Software FoundationSitio web oficial
Información general
Extensiones comunes
.py, .pyc, .pyd, .pyo, .pyw, .pyz, .pyi
Paradigma
Multiparadigma: orientado a objetos, imperativo, funcional, reflexivo
Apareció en
1991
Diseñado por
Guido van Rossum
Última versión estable
3.10.5[1]​ (6 de junio de 2022 (1 mes y 6 días))
Sistema de tipos
Fuertemente tipado, dinámico
Implementaciones
CPython, IronPython, Jython, Python for S60, PyPy, ActivePython, Unladen Swallow
Dialectos
Stackless Python, RPython
Influido por
ABC, ALGOL 68, C, Haskell, Icon, Lisp, Modula-3, Perl, Smalltalk, Java
Ha influido a
Boo, Cobra, D, Falcon, Genie, Groovy, Ruby, JavaScript, Cython, Go Latino
Sistema operativo
Multiplataforma
Licencia
Python Software Foundation License
[editar datos en Wikidata]


Podríamos reestructurar el contenido de forma que sea más legible.

Como cada `tr` tiene en teoría dos columnas, una con la cabecera `th` a la izquierda y el texto `td` a la derecha, podemos usar los índices para formatear el texto de salida:

In [50]:
tr_tags = soup.select(".infobox tr")

for tr_tag in tr_tags:
    th_tags = tr_tag.select("th") #Si tiene cabeceras
    td_tags = tr_tag.select("td") #Si tiene datos
    if len(th_tags) > 0 and len(td_tags) > 0: 
        print(f"{th_tags[0].getText().strip()}: {td_tags[0].getText().strip()}") #strip para borar espacios delante y detrás

Extensiones comunes: .py, .pyc, .pyd, .pyo, .pyw, .pyz, .pyi
Paradigma: Multiparadigma: orientado a objetos, imperativo, funcional, reflexivo
Apareció en: 1991
Diseñado por: Guido van Rossum
Última versión estable: 3.10.5[1]​ (6 de junio de 2022 (1 mes y 6 días))
Sistema de tipos: Fuertemente tipado, dinámico
Implementaciones: CPython, IronPython, Jython, Python for S60, PyPy, ActivePython, Unladen Swallow
Dialectos: Stackless Python, RPython
Influido por: ABC, ALGOL 68, C, Haskell, Icon, Lisp, Modula-3, Perl, Smalltalk, Java
Ha influido a: Boo, Cobra, D, Falcon, Genie, Groovy, Ruby, JavaScript, Cython, Go Latino
Sistema operativo: Multiplataforma
Licencia: Python Software Foundation License


## Scrapeando una imagen

Por último nos quedó pendiente el logo del `infobox`, si inspeccionamos la imagen veremos que tiene un tag `img` con muchos atributos:

```html
<img alt="Python-logo-notext.svg" src="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/100px-Python-logo-notext.svg.png" decoding="async" width="100" height="100" srcset="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/150px-Python-logo-notext.svg.png 1.5x, //web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/200px-Python-logo-notext.svg.png 2x" data-file-width="110" data-file-height="110">
```

Es un poco difícil recuperar este elemento porque no tiene un identificador único, pero...

¿No se encuentra la imagen principal siempre en la tabla `infobox`? Pues vamos a utilizarla para recuperar la primera imagen en su interior:

In [51]:
img = soup.select(".infobox img")[0] #En infobox buscamos la imagen img y 0 busca la primer imagen

print(img)

<img alt="Python-logo-notext.svg" data-file-height="110" data-file-width="110" decoding="async" height="100" src="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/100px-Python-logo-notext.svg.png" srcset="//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/150px-Python-logo-notext.svg.png 1.5x, //web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/200px-Python-logo-notext.svg.png 2x" width="100"/>


Solo tenemos que recuperar el enlace de la imagen en su atributo src:

In [52]:
print(img['src'])

//web.archive.org/web/20220722211457im_/https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/100px-Python-logo-notext.svg.png


Utilizando el propio módulo `requests` podemos guardar la imagen en el directorio de este mismo notebook con el nombre que queramos:

In [53]:
# Ponemos el protocolo https: delante porque el enlace no lo incluye
response = requests.get(f"https:{img['src']}")

if response.status_code == 200:
    with open("image.png", 'wb') as f:
        f.write(response.content)

Si no falla en principio es que se ha descargado, podemos mostrar la imagen en markdown con un simple código:

```markdown
![](image.png)
```

![](image.png)

En la [documentación oficial](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) está toda la información sobre `bs4` y el manejo de la jerarquía, también hay [más ejemplos sobre el uso de select](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors).

In [54]:
import requests
from bs4 import BeautifulSoup

req = requests.get("https://quotes.toscrape.com")
soup = BeautifulSoup(req.text)

# Buscamos las citas de la portada
quotes_tags = soup.select("div.quote")
for quote_tag in quotes_tags:
    # Buscamos el texto
    print(quote_tag.select("span.text")[0].getText())
    # Buscamos el autor
    print(quote_tag.select("small.author")[0].getText())
    # Buscamos las etiquetas
    for tag in quote_tag.select("div.tags a.tag"):
        print(tag.getText(), end=" ")
    # Salto de línea para separar las citas
    print("\n")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
abilities choices 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
aliteracy books classic humor 

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
be-yourself inspirational 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 



Podríamos adaptar este código a una función que a partir de una porción de la URL almacene mediante diccionarios las citas:

In [55]:
def scrap_quotes(url=""):
    domain = "https://quotes.toscrape.com"
    req = requests.get(f"{domain}{url}")
    soup = BeautifulSoup(req.text)

    # Lista para almacenar diccionarios que contendrán datos de las citas
    quotes = []
    # Buscamos las citas de la portada
    quotes_tags = soup.select("div.quote")
    for quote_tag in quotes_tags:
        # Creamos un diccionario vacío
        quote = {}
        # Almacenamos los diferentes campos en el diccinario
        quote['text'] = quote_tag.select("span.text")[0].getText()
        quote['author'] = quote_tag.select("small.author")[0].getText()
        quote['tags'] = []
        for tag in quote_tag.select("div.tags a.tag"):
            quote['tags'].append(tag.getText())
        # Añadimos el diccionario con la cita a la lista
        quotes.append(quote)
    # Devolvemos las citas scrapeadas
    return quotes

quotes = scrap_quotes()

for quote in quotes:
    print(quote["text"])
    print(quote["author"])
    for tag in quote["tags"]:
        print(tag, end=" ")
    print("\n")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
abilities choices 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
aliteracy books classic humor 

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
be-yourself inspirational 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 



La clave es utilizar nuestra función de forma recursiva detectando si la página tiene el enlace **Next** y cargando la siguiente página de manera que podamos. Veamos cómo extraer el enlace con la siguiente página si la hay:

In [56]:
domain = "https://quotes.toscrape.com"
req = requests.get(domain)
soup = BeautifulSoup(req.text)

# Buscamos el enlace en el tag li con clase next
link_tag = soup.select("li.next a")
# Si hay como mínimo un enlace extraemos su href relativo sumado al dominio
if len(link_tag) > 0:
    next_url = link_tag[0]['href']
    print(next_url)

/page/2/


Podemos integrar este código en nuestra función `scrap_quotes` para devolver no solo las citas de la página, sino también si hay una página siguiente:

In [57]:
def scrap_quotes(url=""):
    domain = "https://quotes.toscrape.com"
    req = requests.get(f"{domain}{url}")
    soup = BeautifulSoup(req.text)

    # Lista para almacenar diccionarios que contendrán datos de las citas
    quotes = []
    # Buscamos las citas de la portada
    quotes_tags = soup.select("div.quote")
    for quote_tag in quotes_tags:
        # Creamos un diccionario vacío
        quote = {}
        # Almacenamos los diferentes campos en el diccinario
        quote['text'] = quote_tag.select("span.text")[0].getText()
        quote['author'] = quote_tag.select("small.author")[0].getText()
        quote['tags'] = []
        for tag in quote_tag.select("div.tags a.tag"):
            quote['tags'].append(tag.getText())
        # Añadimos el diccionario con la cita a la lista
        quotes.append(quote)
        
    # Buscamos el enlace en el tag li con clase next
    next_url = None
    link_tag = soup.select("li.next a")
    # Si hay como mínimo un enlace extraemos su href relativo sumado al dominio
    if len(link_tag) > 0:
        next_url = link_tag[0]['href']
    
    # Imprimiros un mensaje informativo
    print(f"Página {domain}{url}, {len(quotes)} citas scrapeadas.")
    
    # Devolvemos las citas scrapeadas y la siguiente página, que puede ser None
    return quotes, next_url

quotes, next_url = scrap_quotes()

print() # Espacio en blanco
print(next_url)

Página https://quotes.toscrape.com, 10 citas scrapeadas.

/page/2/


Vamos a implementar una función que scrapee todas las páginas mientras haya una siguente o, alternativamente, podemos establecer un límite para optimizar el proceso y no saturar al servidor:

In [58]:
def scrap_site(limit=2):
    # Definimos una lista global para almacenar todas las citas
    all_quotes = []
    # Definimos la siguiente URL que irá cambiando (inicialmente es el dominio raíz)
    next_url = "" 
    # Iniciamos un bucle infinito
    while 1:
        # Scrapeamos la página, guardamos las citas scrapeadas y la siguiente página
        quotes, next_url = scrap_quotes(next_url)
        # Añadimos las citas scrapeadas a la lista global
        all_quotes += quotes
        # Restamos 1 al limite 
        limit -= 1
        # Si lo superamos o no hay siguiente página finalizamos la función
        if limit == 0 or next_url == None:
            # Finalizamos la función
            return all_quotes

quotes = scrap_site()

print() # Espacio en blanco
for quote in quotes:
    print(quote["text"])
    print(quote["author"])
    for tag in quote["tags"]:
        print(tag, end=" ")
    print("\n")

Página https://quotes.toscrape.com, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/2/, 10 citas scrapeadas.

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
abilities choices 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
aliteracy books classic humor 

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
be-yourself inspirational 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood

Con esto logramos una función capaz de scrapear todas las citas de la página por defecto limitado a 2 páginas.

## Implementando la clase Citas

Vamos a seguir con la clase `Citas` y el método `scrapear` pero crearemos un fichero donde almacenar todas las citas.

### Guardado en fichero

Solo generaremos el fichero si ejecutamos el método `scrapear`, los demás métodos `lista`, `etiqueta` y `autor` analizarán el contenido del fichero volcado en la memoria, pero nunca scrapearán nada directamente.

Después de valorarlo he decidido utilizar un CSV. Lo único que nos dará algún problema es guardar una lista como un campo del registro, pero podemos recuperarla evaluándola de nuevo, ya veréis:

In [59]:
import csv

class Citas:
    
    # Variable de clase para almacenar las citas en la memoria
    quotes = []
    
    @staticmethod
    def scrapear():
        # Scrapeamos todas las citas, ponemos un límite pequeño para hacer pruebas
        Citas.quotes = scrap_site(limit=2)
        # Guardamos las citas scrapeadas en un fichero CSV volcándolas de la lista de dicts
        with open("quotes.csv", "w") as file:
            # Definimos el objeto para escribir con las cabeceras de los campos 
            writer = csv.DictWriter(file, fieldnames=["text", "author", "tags"])
            # Escribimos las cabeceras
            writer.writeheader()
            # Escribimos cada cita en la memoria en el fichero
            for quote in Citas.quotes:
                writer.writerow(quote)
            
Citas.scrapear()

Página https://quotes.toscrape.com, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/2/, 10 citas scrapeadas.


En este punto deberíamos tener un fichero `quotes.csv` con todas las citas, lo que podríamos hacer es cargar en la memoria todas las citas del fichero en caso de que éste exista. De paso podemos implementar el método `lista` para consultarlas:

In [60]:
import os
import csv

class Citas:
    
    # Variable de clase para almacenar las citas en la memoria
    quotes = []
    
    # Recuperamos las citas en la memoria si existe el fichero quotes.csv
    if os.path.exists("quotes.csv"):
        with open("quotes.csv", "r") as file:
            data = csv.DictReader(file)
            for quote in data:
                # La lista es una cadena, hay que reevaluarla
                quote['tags'] = eval(quote['tags'])
                quotes.append(quote)
    
    @staticmethod
    def scrapear():
        # Scrapeamos todas las citas, ponemos un límite pequeño para hacer pruebas
        Citas.quotes = scrap_site(limit=2)
        # Guardamos las citas scrapeadas en un fichero CSV volcándolas de la lista de dicts
        with open("quotes.csv", "w") as file:
            # Definimos el objeto para escribir con las cabeceras de los campos 
            writer = csv.DictWriter(file, fieldnames=["text", "author", "tags"])
            # Escribimos las cabeceras
            writer.writeheader()
            # Escribimos cada cita en la memoria en el fichero
            for quote in Citas.quotes:
                writer.writerow(quote)
            
    @staticmethod
    def listar(limite=10):
        for quote in Citas.quotes[:limite]:
            print(quote["text"])
            print(quote["author"])
            for tag in quote["tags"]:
                print(tag, end=" ")
            print("\n")
            
Citas.listar(5)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
abilities choices 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
aliteracy books classic humor 

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
be-yourself inspirational 



### Filtro por etiqueta y autor

Ya solo nos falta implementar los métodos de filtrado por etiqueta y autor, es muy fácil porque solo tenemos que recorrer las citas y comprobar si concuerdan con los valores que pasamos a los métodos:

In [61]:
import os
import csv

class Citas:
    
    # Variable de clase para almacenar las citas en la memoria
    quotes = []
    
    # Recuperamos las citas en la memoria si existe el fichero quotes.csv
    if os.path.exists("quotes.csv"):
        with open("quotes.csv", "r") as file:
            data = csv.DictReader(file)
            for quote in data:
                # La lista es una cadena, hay que reevaluarla
                quote['tags'] = eval(quote['tags'])
                quotes.append(quote)
    
    @staticmethod
    def scrapear():
        # Scrapeamos todas las citas, ponemos un límite pequeño para hacer pruebas
        Citas.quotes = scrap_site(limit=2)
        # Guardamos las citas scrapeadas en un fichero CSV volcándolas de la lista de dicts
        with open("quotes.csv", "w") as file:
            writer = csv.DictWriter(file, fieldnames=["text", "author", "tags"])
            writer.writeheader()
            for quote in Citas.quotes:
                writer.writerow(quote)
            
    @staticmethod
    def listar(limite=10):
        for quote in Citas.quotes[:limite]:
            print(quote["text"])
            print(quote["author"])
            for tag in quote["tags"]:
                print(tag, end=" ")
            print("\n")

    @staticmethod
    def etiqueta(nombre=""):
        for quote in Citas.quotes:
            if nombre in quote["tags"]:
                print(quote["text"])
                print(quote["author"])
                for tag in quote["tags"]:
                    print(tag, end=" ")
                print("\n")
                
    @staticmethod
    def autor(nombre=""):
        for quote in Citas.quotes:
            if nombre == quote["author"]:
                print(quote["text"])
                print(quote["author"])
                for tag in quote["tags"]:
                    print(tag, end=" ")
                print("\n")

Veamos cuantas citas tenemos con el tag **love**:

In [62]:
Citas.etiqueta("love")

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 

“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate to say it, most of them - actually pretty much all of them are going to break your heart, but you can't give up because if you give up, you'll never find your soulmate. You'll never find that half who makes you whole and that goes for everything. Just because you fail once, doesn't mean you're gonna fail at everything. Keep trying, hold on, and always, always, always believe in yourself, beca

Y del autor **Albert Einstein**:

In [63]:
Citas.autor("Albert Einstein")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“If you can't explain it to a six year old, you don't understand it yourself.”
Albert Einstein
simplicity understand 



## Scrapeo de la web completa

El programa está limitado a las 2 primeras páginas, se reescribirá el código con un límite muy grande que garantice un scrapeo completo de la web:

In [65]:
import os
import csv
import requests
from bs4 import BeautifulSoup


def scrap_quotes(url=""):
    domain = "https://quotes.toscrape.com"
    req = requests.get(f"{domain}{url}")
    soup = BeautifulSoup(req.text)
    
    quotes = []
    quotes_tags = soup.select("div.quote")
    for quote_tag in quotes_tags:
        quote = {}
        quote['text'] = quote_tag.select("span.text")[0].getText()
        quote['author'] = quote_tag.select("small.author")[0].getText()
        quote['tags'] = []
        for tag in quote_tag.select("div.tags a.tag"):
            quote['tags'].append(tag.getText())
        quotes.append(quote)
        
    next_url = None
    link_tag = soup.select("li.next a")
    if len(link_tag) > 0:
        next_url = link_tag[0]['href']
        
    print(f"Página {domain}{url}, {len(quotes)} citas scrapeadas.")
        
    return quotes, next_url


def scrap_site(limit=2):
    all_quotes = []
    next_url = "" 
    while 1:
        quotes, next_url = scrap_quotes(next_url)
        all_quotes += quotes
        limit -= 1
        if limit == 0 or next_url == None:
            return all_quotes

        
class Citas:
    quotes = []
    
    if os.path.exists("quotes.csv"):
        with open("quotes.csv", "r") as file:
            data = csv.DictReader(file)
            for quote in data:
                quote['tags'] = eval(quote['tags'])
                quotes.append(quote)
    
    @staticmethod
    def scrapear():
        Citas.quotes = scrap_site(limit=99) # <--- LIMITE MUY GRANDE
        with open("quotes.csv", "w") as file:
            writer = csv.DictWriter(file, fieldnames=["text", "author", "tags"])
            writer.writeheader()
            for quote in Citas.quotes:
                writer.writerow(quote)
            
    @staticmethod
    def listar(limite=10):
        for quote in Citas.quotes[:limite]:
            print(quote["text"])
            print(quote["author"])
            for tag in quote["tags"]:
                print(tag, end=" ")
            print("\n")

    @staticmethod
    def etiqueta(nombre=""):
        for quote in Citas.quotes:
            if nombre in quote["tags"]:
                print(quote["text"])
                print(quote["author"])
                for tag in quote["tags"]:
                    print(tag, end=" ")
                print("\n")
                
    @staticmethod
    def autor(nombre=""):
        for quote in Citas.quotes:
            if nombre == quote["author"]:
                print(quote["text"])
                print(quote["author"])
                for tag in quote["tags"]:
                    print(tag, end=" ")
                print("\n")

Vamos a ejecutar el scrapeo completo:

In [66]:
Citas.scrapear()

Página https://quotes.toscrape.com, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/2/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/3/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/4/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/5/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/6/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/7/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/8/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/9/, 10 citas scrapeadas.
Página https://quotes.toscrape.com/page/10/, 10 citas scrapeadas.


Veamos cuantas citas encuentra ahora con el tag **love**:

In [67]:
Citas.etiqueta("love")

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
life love 

“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate to say it, most of them - actually pretty much all of them are going to break your heart, but you can't give up because if you give up, you'll never find your soulmate. You'll never find that half who makes you whole and that goes for everything. Just because you fail once, doesn't mean you're gonna fail at everything. Keep trying, hold on, and always, always, always believe in yourself, beca

Y cuantas del autor **Albert Einstein**:

In [68]:
Citas.autor("Albert Einstein")

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
change deep-thoughts thinking world 

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
inspirational life live miracle miracles 

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
adulthood success value 

“If you can't explain it to a six year old, you don't understand it yourself.”
Albert Einstein
simplicity understand 

“If you want your children to be intelligent, read them fairy tales. If you want them to be more intelligent, read them more fairy tales.”
Albert Einstein
children fairy-tales 

“Logic will get you from A to Z; imagination will get you everywhere.”
Albert Einstein
imagination 

“Any fool can know. The point is to understand.”
Albert Einstein
knowledge learning understanding wisdom 

“Life is like riding a b

Parece que todo funciona correctamente y podemos hacer tantas consultas como queramos sin repetir una y otra vez el proceso de scrapeo. En la práctica podríamos configurar un script que scrapee la página una vez al día para tener el fichero CSV sincronizado.

En cualquier caso con esto acabamos este ejemplo y también la sección, espero que hayáis aprendido mucho.


Referencias
- C. Hector. (2024). [Curso Maestro de Python: De Cero a Programador Todoterreno](https://www.udemy.com/course/python-3-al-completo-desde-cero/?couponCode=ACCAGE0923).
